# Submitting batch prediction jobs to Cloud AI Platform Prediction

In [ ]:
!gsutil ls gs://workshop-datasets/cifar10

In [1]:
import absl
import tensorflow as tf
import tensorflow_hub as hub

import numpy as np

from tensorflow.python.framework.ops import disable_eager_execution

tf.__version__

import IPython.display as display

In [2]:
#disable_eager_execution()
tf.executing_eagerly()

True

In [3]:
inputs = tf.keras.layers.Input(shape=(32, 32, 3))
d1 = tf.keras.layers.Conv2D(64, kernel_size=3, activation='relu')(inputs)
d2 = tf.keras.layers.Conv2D(32, kernel_size=3, activation='relu')(d1)
d3 = tf.keras.layers.Flatten()(d2)
outputs = tf.keras.layers.Dense(10, activation='softmax')(d3)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [4]:
model.compile(
    optimizer=tf.keras.optimizers.SGD(lr=0.0001, momentum=0.9),
    loss='categorical_crossentropy',
    metrics=[tf.keras.metrics.CategoricalAccuracy(name='accuracy')])

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 30, 30, 64)        1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 32)        18464     
_________________________________________________________________
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
dense (Dense)                (None, 10)                250890    
Total params: 271,146
Trainable params: 271,146
Non-trainable params: 0
_________________________________________________________________


In [5]:
image_features = {
    'image_raw': tf.io.FixedLenFeature([], tf.string),
    'label': tf.io.FixedLenFeature([], tf.int64)
}

def parse_image(example_proto):
    parsed_image = tf.io.parse_single_example(example_proto, image_features)
    image_str = parsed_image['image_raw']
    label = parsed_image['label']
    image = tf.image.decode_image(image_str, channels=3)
    image = tf.reshape(image, (32, 32, 3))
    image = tf.cast(image, tf.float32) / 255
#    label = tf.one_hot(label_id, 10)
    return image, label
    


In [11]:
train_files = ['gs://workshop-datasets/cifar10/train.tfrecord']
validation_files = ['gs://workshop-datasets/cifar10/test.tfrecord']
train_dataset = tf.data.TFRecordDataset(train_files)
validation_dataset = tf.data.TFRecordDataset(validation_files)


train_dataset = (train_dataset.map(parse_image)
                 .batch(64, drop_remainder=True))

In [12]:
for image, label in train_dataset.take(1):
    print((image.shape, label.shape))

(TensorShape([64, 32, 32, 3]), TensorShape([64]))


In [13]:
validation_dataset = (validation_dataset.map(parse_image)
                 .batch(64, drop_remainder=True))

In [14]:
model.fit(train_dataset,
         epochs=10,
         validation_data=validation_dataset,
         validation_freq=1)

Epoch 1/10
    691/Unknown - 52s 75ms/step - loss: nan - accuracy: 0.9774

KeyboardInterrupt: 

In [ ]:
for parsed_image in parsed_image_dataset.take(2):
    print(parsed_image)

In [ ]:
dataset = tf.data.TFRecordDataset(tfrecord_files)
#parsed_image_dataset = dataset.map(parse_image)

for parsed_image in parsed_image_dataset.take(2):
    image_raw  = parsed_image['image_raw'].numpy()
    display.display(display.Image(data=image_raw))

In [ ]:
tfrecord_files = ['gs://workshop-datasets/cifar10/test.tfrecord']

image_features = {
    'image_raw': tf.io.FixedLenFeature([], tf.string),
    'label': tf.io.FixedLenFeature([], tf.int64)
}

